In [ ]:
import cv2 # pip3 install opencv-python
import numpy as np
import os
from random import shuffle
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
train_dir = '../input/train/'
test_dir = '../input/test/'
img_size = 50
lr = 1e-3

In [ ]:
# one hot encoding
def label_img(img):
    word_label = img.split('.')[-3]
    if word_label == 'cat': return [1,0]
    elif word_label == 'dog': return [0,1]

In [ ]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(train_dir)):
        if (not img.endswith('.jpg')):
            continue
        label = label_img(img)
        path = os.path.join(train_dir, img)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  # greyscale
        img = cv2.resize(img, (img_size, img_size) )  # unite the figure size
        training_data.append([np.array(img), np.array(label)])
    shuffle(training_data)
    return training_data

In [ ]:
train_data = create_train_data()

In [ ]:
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(test_dir)):
        if (not img.endswith('.jpg')):
            continue
        path = os.path.join(test_dir,img)
        img_num = img.split('.')[0]
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (img_size, img_size))
        testing_data.append([np.array(img), img_num]) # no label
        
    shuffle(testing_data)
    return testing_data

In [ ]:
test_data = process_test_data()

In [ ]:
import tflearn # need to install tensorflow first
from tflearn.layers.conv import conv_2d, max_pool_2d  # 2d_CNN and max pooling
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression # cross entropy

In [ ]:
import tensorflow as tf
tf.reset_default_graph() # need to reset the default graph for multiple running

In [ ]:
convnet = input_data(shape = [None, img_size, img_size, 1], name = 'input')

**5xCNN layers, 5xMax pooling layer, 5x5xChannel**

In [ ]:
convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

**2xFully connected layers and a predict layer**

In [ ]:
convnet = fully_connected(convnet, 1024, activation = 'relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate = lr, loss='categorical_crossentropy', name='targets')

**Set up model, trainning set and test set**

In [ ]:
model = tflearn.DNN(convnet, tensorboard_dir='log')
train = train_data[:-500]
test = train_data[-500:]

In [ ]:
X = np.array([i[0] for i in train], dtype=np.float64).reshape(-1, img_size, img_size, 1)
y = np.array([i[1] for i in train], dtype=np.float64)
Xtest = np.array([i[0] for i in test], dtype=np.float64).reshape(-1, img_size, img_size, 1)
ytest = np.array([i[1] for i in test], dtype=np.float64)

**Model train**

In [ ]:
model.fit({'input': X}, {'targets': y}, n_epoch=3, validation_set=({'input': Xtest}, {'targets': ytest}), snapshot_step=500, show_metric=True, run_id='model' )

**Predict on test dataset and check the results**

In [ ]:
fig = plt.figure()
for num,data in enumerate(test_data[:16]):
    img_num = data[1]
    img_data = data[0]
    y = fig.add_subplot(4, 4, num+1)
    orig = img_data
    data = img_data.reshape(img_size, img_size, 1)
    model_out = model.predict([data])[0]
    if np.argmax(model_out) == 1: 
        label = 'Dog'
    else: 
        label = 'Cat'
    y.imshow(orig, cmap='gray')
    plt.title(label)
    y.axes.get_xaxis().set_visible(False)
    y.axes.get_yaxis().set_visible(False)
    
plt.tight_layout()
plt.show()